In [2]:
from bs4 import BeautifulSoup
from lxml import etree
import json
import requests
import pandas as pd
import numpy as np
import re
import os
from datetime import datetime

___
# **Easy**

In [3]:
class WebScrapingEasy:
    def __init__(self, path):
        self.path = path
    def cleanPath(self):
        txt = self.path.lower().split(' > ')
        txt = [i.replace(' ', '-') for i in txt]
        txt = '/'.join(txt)
        txt = txt.replace('ñ', 'n')
        txt = re.sub(r'[àáâãäå]', 'a', txt)
        txt = re.sub(r'[èéêë]'  , 'e', txt)
        txt = re.sub(r'[ìíîï]'  , 'i', txt)
        txt = re.sub(r'[òóôõö]' , 'o', txt)
        txt = re.sub(r'[ùúûü]'  , 'u', txt)
        return txt
    def getValues(self, Soup):
        Titulos = Soup.find_all('div', attrs = {'class' : 'dpr_product-name'})
        Titulos = [i.text for i in Titulos]
        Precios = Soup.find_all('div', attrs = {'class' : 'dpr_listprice'})
        Precios = [i.text for i in Precios]
        Precios = [i.replace('$', '') for i in Precios]
        Precios = [i.replace(',', '.') for i in Precios]
        URLs = Soup.find_all('a', attrs = {'class' : 'dpr_listname'})
        URLs = [i.get('href') for i in URLs]
        URLs = ['https://www.easy.com.co' + i for i in URLs]
        return Titulos, Precios, URLs
    def createDF(self, Titles, Prices, URLs):
        df = pd.DataFrame({'URL': URLs, 'Producto': Titles, 'Precio': Prices})
        df['Origen'] = "Easy"
        df.to_csv('Support Files/Easy.csv', index = False)
        df.to_csv('Support Files/Easy.txt', sep = '|', index = False)
        return df
    def controller(self):
        Website = 'https://www.easy.com.co/c/'+self.cleanPath()+'/'
        listTitulos = []
        listPrecios = []
        listURLs = []
        j = 1
        while True:
            print(f"Página Número: {j} | Link: {Website}")
            Result  = requests.get(Website)
            if Result.status_code == 200:
                Content = Result.content
                Soup = BeautifulSoup(Content, 'html.parser') # 'lxml'
                # print(Soup.prettify())
                Dom  = etree.HTML(str(Soup))
                Next = Dom.xpath('//div[@class = "top-listado-producto pr"][1]/div[@class = "paginador"]/div[@class = "page_nav_grid"]/ul/li/a')
                Titulos, Precios, URLs = self.getValues(Soup)
                listTitulos.extend(Titulos)
                listPrecios.extend(Precios)
                listURLs.extend(URLs)
                if Next == []:
                    break
                else:
                    if j == 1:
                        # nStart = int(Next[0].text)
                        nEnd   = int(Next[-2].text)
                        # print(nStart, nEnd)
                    if j == nEnd:
                        break
                    j += 1
                    Website = 'https://www.easy.com.co'+Next[-1].get('href')
            else:
                print(f"Sin respuesta por parte del servidor. Fallo al tratar de visitar la página web:\n\t--> {Website}"
                )
                break
        return self.createDF(listTitulos, listPrecios, listURLs)

___
# **Homecenter**

In [4]:
class WebScrapingHomeCenter:
    def __init__(self, path):
        self.path = path
    def getCategories(self):
        filePath = 'Code/CategoriesList_HomeCenter.pkl'
        if os.path.isfile(filePath) == False:
            print(
                "En el directorio no se encuentra el archivo:\n\
                \tCategoriesList_HomeCenter.pkl"
            )
        else:
            return pd.read_pickle("Code/CategoriesList_HomeCenter.pkl")
    def checkStatus(self, url):
        try:
            Result = requests.get(url)
            status = "Conexión Valida" if Result.status_code == 200 else "Conexión sin Respuesta"
        except:
            status = "Conexión sin Respuesta"
        hora = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")
        return (status, hora)
    def updateURL(self, df, all):
        if all:
            df['infoStatus'] = df['URL'].apply(self.checkStatus)
            df[['Status', 'Status_Date']] = pd.DataFrame(
                df['infoStatus'].tolist(), index = df.index
            )
            df.drop(['infoStatus'], axis = 1, inplace = True)
            print("___________________________________________________________")
        else:
            newURL = input(
                f"Ingrese la nueva URL para la categoría: {self.path}"
            )
            x = self.checkStatus(newURL)
            df.loc[df["Categoria"] == self.path, ['URL', 'Status', 'Status_Date']] = [newURL, x[0], x[1]]
        df.to_pickle("Code/CategoriesList_HomeCenter.pkl")
    def addCategories(self, df):
        Input_Cat = self.path
        Input_URL = input("Ingrese la URL de la nueva categoría: ")
        newRow = {
            'Categoria'  : Input_Cat,
            'URL'        : Input_URL,
            'Status'     : 'Nueva Categoría',
            'Status_Date': (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")
        }
        df = df.append(newRow, ignore_index = True)
        df.to_pickle("Code/CategoriesList_HomeCenter.pkl")
        return Input_URL
    def getValues(self, Soup, Dom):
        Titulos = Soup.find_all('h2', attrs = {'class' : 'jsx-2974854745 product-title'})
        if Titulos == []:
            Titulos = Soup.find_all('div', attrs = {'class' : 'jsx-110785930 title-name'})
        Titulos = [i.text for i in Titulos]

        Precios = Dom.xpath('//div[@class = "jsx-2974854745 desktop-price-cart-btn"]//div[contains(@class, "jsx-344173702 main")]/div[@class = "jsx-4135487716 price jsx-175035124"]/span[@class = "jsx-4135487716"]')
        if Precios == []:
            Precios = Dom.xpath('//div[@class = "jsx-110785930 price-container"]//div[contains(@class, "jsx-344173702 main")]/div[@class = "jsx-4135487716 price jsx-175035124"]/span[@class = "jsx-4135487716"]')
        Precios = [i.text for i in Precios]
        Precios = [i.replace('$', '') for i in Precios]

        URLs = Soup.find_all('a', attrs = {'class' : 'jsx-2974854745'})
        if URLs == []:
            URLs = Soup.find_all('a', attrs = {'id' : 'testId-Link-title-pdp-link'})
        URLs = [i.get('href') for i in URLs]
        URLs = ['https://www.homecenter.com.co' + i for i in URLs]
        if len(Titulos) == len(Precios) == len(URLs):
            return Titulos, Precios, URLs
        else:
            print(
                f'''
                La longitud de los tres arreglos no coincide:
                \t--> Titulos tiene una longitud de {len(Titulos)}
                \t--> Precios tiene una longitud de {len(Precios)}
                \t--> URLs tiene una longitud de {len(URLs)}
                '''
            )
    def createDF(self, Titles, Prices, URLs):
        df = pd.DataFrame({'URL': URLs, 'Producto': Titles, 'Precio': Prices})
        df['Origen'] = "HomeCenter"
        df.to_csv('Support Files/HomeCenter.csv', index = False)
        df.to_csv('Support Files/HomeCenter.txt', sep = '|', index = False)
        return df
    def controller(self):
        df = self.getCategories()
        # Monday = 0 | Sunday = 6
        if (datetime.now()).weekday() == -1:
            print(
                "Hoy se realizará la actualización del status de las conexiones\
                \nEsto puede tomar un tiempo, por favor espere ..."
            )
            self.updateURL(df, all = True)
            print('¡Actualización finalizada con éxito!')
        # ----------------------------------------------------------------------
        try:
            Website = df[df['Categoria'].str.lower() == self.path.lower()]['URL'].values[0]
            if self.checkStatus(Website)[0] == "Conexión sin Respuesta":
                self.updateURL(df, all = False)
                Website = df[df['Categoria'].str.lower() == self.path.lower()]['URL'].values[0]
        except:
            Website = self.addCategories(df)
        print(Website)
        listTitulos = []
        listPrecios = []
        listURLs = []
        j = 1
        while True:
            print(f"Página Número: {j} | Link: {Website}")
            Result = requests.get(Website)
            if Result.status_code == 200:
                Content = Result.content
                Soup = BeautifulSoup(Content, 'html.parser')
                Dom  = etree.HTML(str(Soup))

                Titulos, Precios, URLs = self.getValues(Soup, Dom)
                listTitulos.extend(Titulos)
                listPrecios.extend(Precios)
                listURLs.extend(URLs)

                Next = Dom.xpath('//div[@class = "jsx-861649955 tablet-desktop media-component"]//div[@class = "jsx-4278284191 scroll-area"]//button[@class = "jsx-4278284191"]')
                Next = [int(i.text) for i in Next]
                if Next == []:
                    break
                else:
                    if j == 1:
                        nEnd = max(Next)
                        base = re.sub(r".$", "?currentpage=", Website)
                    if j == nEnd:
                        break
                    j += 1
                    Website = base + str(j)
            else:
                print(f"Sin respuesta por parte del servidor. Fallo al tratar de visitar la página web:\n\t--> {Website}"
                )
                break
        return self.createDF(listTitulos, listPrecios, listURLs)

___
# **Mercado Libre**

In [5]:
class WebScrapingMercadoLibre:
    def __init__(self, product, limit, infoExtra):
        self.product   = product
        self.limit     = limit
        self.infoExtra = infoExtra
    def cleanPath(self):
        txt = self.product.lower()
        txt = txt.replace(' ', '-')
        txt = txt.replace('ñ', 'n')
        txt = re.sub(r'[àáâãäå]', 'a', txt)
        txt = re.sub(r'[èéêë]'  , 'e', txt)
        txt = re.sub(r'[ìíîï]'  , 'i', txt)
        txt = re.sub(r'[òóôõö]' , 'o', txt)
        txt = re.sub(r'[ùúûü]'  , 'u', txt)
        return txt
    def getInfoExtra(self, URL):
        Result = requests.get(URL)
        if Result.status_code == 200:
            Content = Result.content
            Soup = BeautifulSoup(Content, 'html.parser')
            Dom  = etree.HTML(str(Soup))

            Path = Soup.find_all('a', attrs = {'class' : 'andes-breadcrumb__link'})
            Path   = [i.text for i in Path]
            Path   = ' > '.join(Path)
            Trade  = Soup.find('span', attrs = {'class' : 'ui-pdp-subtitle'})
            Trade  = int(re.findall(r'\d+', Trade.text)[0])
            # X = Soup.find('p', attrs = {'class' : 'ui-review-capability__rating__average ui-review-capability__rating__average--desktop'})
            Rating = Dom.xpath('//p[contains(@class, "ui-review-capability__rating__average")]')
            try:
                Rating = [float(i.text) for i in Rating][0]
            except:
                Rating = np.nan
            Envio = Dom.xpath('//p[contains(@class, "ui-pdp-color--GREEN ui-pdp-family--REGULAR ui-pdp-media__title")]')
            Envio = [i.text for i in Envio]
            if Envio != []:
                Envio = True if Envio != ['Paga en cuotas sin interés'] else False
            else:
                Envio = False

            return Path, Rating, Trade, Envio
        else:
            print(f'Página del producto en cuestión NO encontrada, no se retorna información extra de este\nURL: {URL}')
            return np.nan, np.nan, np.nan, False
    def getValues(self, Soup, Dom):
        Titulos = Soup.find_all('h2', attrs = {'class' : 'ui-search-item__title shops__item-title'})
        Titulos = [i.text for i in Titulos]
        Precios = Dom.xpath('//div[@class = "ui-search-price ui-search-price--size-medium shops__price"]//div[@class = "ui-search-price__second-line shops__price-second-line"]//span[@class = "price-tag-amount"]/span[2]')
        Precios = [i.text for i in Precios]
        URLs = Soup.find_all('a', attrs = {'class' : 'ui-search-item__group__element shops__items-group-details ui-search-link'})
        URLs = [i.get('href') for i in URLs]
        return Titulos, Precios, URLs
    def createDF(self, Titles, Prices, URLs):
        df = pd.DataFrame({'URL': URLs, 'Producto': Titles, 'Precio': Prices})
        df['Origen'] = "Mercado Libre"
        df.to_csv('Support Files/MercadoLibre.csv', index = False)
        df.to_csv('Support Files/MercadoLibre.txt', sep = '|', index = False)
        return df
    def controller(self):
        Website = 'https://listado.mercadolibre.com.co/' + self.cleanPath()
        listTitulos = []
        listPrecios = []
        listURLs = []
        while True:
            Result  = requests.get(Website)
            if Result.status_code == 200:
                Content = Result.content
                Soup = BeautifulSoup(Content, 'html.parser')
                Dom  = etree.HTML(str(Soup))

                Titulos, Precios, URLs = self.getValues(Soup, Dom)
                listTitulos.extend(Titulos)
                listPrecios.extend(Precios)
                listURLs.extend(URLs)

                nStart = int(
                    Soup.find('span', attrs = {"class" : "andes-pagination__link"}).text
                )
                nEnd = Soup.find('li', attrs = {"class" : "andes-pagination__page-count"})
                nEnd = int(
                    nEnd.text.split(" ")[1]
                )
            else:
                print(f"Sin respuesta por parte del servidor. Fallo al tratar de visitar la página web:\n\t--> {Website}"
                )
                break
            print(f"Visitando la Página Número: {nStart} de {nEnd}")
            print(f"Link: {Website}")
            if len(listTitulos) >= int(self.limit):
                df = self.createDF(
                    listTitulos[:self.limit],
                    listPrecios[:self.limit],
                    listURLs[:self.limit]
                )
                if self.infoExtra:
                    df['Tuple'] = df['URL'].apply(self.getInfoExtra)
                    df[['Categoria', 'Ranting', 'Vendidos', 'EnvíoGratis']] = pd.DataFrame(df['Tuple'].tolist(), index = df.index)
                    df.drop(['Tuple'], axis = 1, inplace = True)
                    df.to_csv('Support Files/MercadoLibre.csv', index = False)
                    df.to_csv('Support Files/MercadoLibre.txt', sep = '|', index = False)
                return df
            if nStart == nEnd:
                break
            Website = Dom.xpath('//div[@class="ui-search-pagination shops__pagination-content"]/ul/li[contains(@class,"--next")]/a')[0].get('href')
        return ""

___
# **AliExpress**

In [6]:
class WebScrapingAliExpress:
    def __init__(self, product, limit):
        self.product = product
        self.limit   = limit
    def cleanPath(self):
        txt = self.product.lower()
        txt = txt.replace(' ', '+')
        txt = txt.replace('ñ', 'n')
        txt = re.sub(r'[àáâãäå]', 'a', txt)
        txt = re.sub(r'[èéêë]'  , 'e', txt)
        txt = re.sub(r'[ìíîï]'  , 'i', txt)
        txt = re.sub(r'[òóôõö]' , 'o', txt)
        txt = re.sub(r'[ùúûü]'  , 'u', txt)
        return txt
    def getConnection(self, search, page):
        url    = "https://es.aliexpress.com/glosearch/api/product"
        urlRaw = 'https://es.aliexpress.com/wholesale?trafficChannel=ppc&d=y&CatId=0&SearchText={}&ltype=premium&SortType=default&page={}'.format(search, page)
        query_string_parameters = {
            'trafficChannel' : 'ppc'    ,
            'd'              : 'y'      ,
            'CatId'          : '0'      ,
            'SearchText'     : search   ,
            'ltype'          : 'premium',
            'SortType'       : 'default',
            'page'           : str(page),
            'origin'         : 'y'
        }
        headers = {
            'accept': 'application/json, text/plain, */*',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'es-419,es;q=0.9',
            'bx-v': '2.2.3',
            'cookie': 'acs_usuc_t=x_csrf=1e83jkp1mo0xf&acs_rt=bc33d50efba94b6fb9b8abfdd1ed9ba6; aeu_cid=6afa8bad078e42a0ba2674ea27a0dc7f-1669256183899-05653-UneMJZVf; xman_t=3tn7B9rqQ8sVaohEod6BPWSDslwwzGlfOVRP0Jhf097sZL4o4wkHoTKRc1iE6pFN; xman_f=umZ8TL4zuB6Cu7fs8HRdn8t0FY8crEiG5MlWTkMhCZsM+AQGWBmKzL9nZEq42c8c2beLnRypHNPhITo1KZMsSJ7p0lL+uQPq9aD/GQ1YcuQXGF3QHc9lBw==; ali_apache_id=33.1.233.210.1669256184316.569493.2; intl_locale=es_ES; cna=+sEFHEWkrHICAbqbhd2ZxuSR; _gid=GA1.2.749679866.1669256188; _gac_UA-17640202-1=1.1669256188.CjwKCAiApvebBhAvEiwAe7mHSGDdjIt-FP4DEcl4R7wgt2qOmzrb6L4hx7iMS6yUI7FudKbj9rZnexoCEEAQAvD_BwE; xlly_s=1; _gcl_aw=GCL.1669256188.CjwKCAiApvebBhAvEiwAe7mHSGDdjIt-FP4DEcl4R7wgt2qOmzrb6L4hx7iMS6yUI7FudKbj9rZnexoCEEAQAvD_BwE; _gcl_dc=GCL.1669256188.CjwKCAiApvebBhAvEiwAe7mHSGDdjIt-FP4DEcl4R7wgt2qOmzrb6L4hx7iMS6yUI7FudKbj9rZnexoCEEAQAvD_BwE; _gcl_au=1.1.1346632231.1669256188; _fbp=fb.1.1669256187874.1553598377; _ym_uid=1669256189713719043; _ym_d=1669256189; _ym_isad=2; ali_apache_track=; ali_apache_tracktmp=; e_id=pt60; XSRF-TOKEN=523a85a2-a51e-4432-94c5-f12b7d10e88b; account_v=1; aep_usuc_f=site=esp&c_tp=COP&region=CO&b_locale=es_ES; AKA_A2=A; xman_us_f=x_locale=es_ES&x_l=0&x_c_chg=0&x_as_i=%7B%22aeuCID%22%3A%226afa8bad078e42a0ba2674ea27a0dc7f-1669256183899-05653-UneMJZVf%22%2C%22affiliateKey%22%3A%22UneMJZVf%22%2C%22channel%22%3A%22PREMINUM%22%2C%22cv%22%3A%222%22%2C%22isCookieCache%22%3A%22N%22%2C%22ms%22%3A%221%22%2C%22pid%22%3A%22178094261%22%2C%22tagtime%22%3A1669256183899%7D&acs_rt=bc33d50efba94b6fb9b8abfdd1ed9ba6; _m_h5_tk=ad7ce0544cae0218ad4f655cae989b47_1669268185710; _m_h5_tk_enc=da927f18e91f29767539ef4469a4bb60; aep_history=keywords%5E%0Akeywords%09%0A%0Aproduct_selloffer%5E%0Aproduct_selloffer%091005003932740121%091005004430757538%091005004448609411; _ym_visorc=b; _gat=1; _ga=GA1.1.276904634.1669256188; RT="z=1&dm=aliexpress.com&si=7be28f9c-9c31-4e03-bcbb-f2c03a5f223c&ss=laumgnum&sl=1&tt=1xt&rl=1&ld=1xv"; JSESSIONID=3D5912BBF3584A85FD8A9F8CDE618A0A; intl_common_forever=oIKdtXaeB7roRyvuv01SfF+3pJQVNbtcTl6z0Tneo9PCfzRKKrVC1A==; _ga_VED1YSGNC7=GS1.1.1669266387.3.1.1669266997.0.0.0; l=eBMP6WIqTBp0yszFBO5wlurza77OyQvfGsPzaNbMiInca6GAtpPTHNCU8sQvSdtjQtfhqetPgOBNgd3681zdgimiTHhHz852MYvM-; tfstk=cTDNBAXAEdpZBVmvyJ2qU424e_POayMmFCEYsEf0YSxWoUVgzsYGkkAk-V3VYZVG.; isg=BHt7DPj9nPyHKaBIjpIDngPsGl_l0I_Sdv4q9206Z3ujzJ-u9aHZIvVK5nSCbOfK',
            'referer': urlRaw,
            'sec-ch-ua': '"Chromium";v="106", "Not.A/Brand";v="24", "Opera GX";v="92"',
            'sec-ch-ua-mobile': '?0',
            'sec-ch-ua-platform': '"Windows"',
            'sec-fetch-dest': 'empty',
            'sec-fetch-mode': 'cors',
            'sec-fetch-site': 'same-origin',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 OPR/92.0.0.0'
        }
        return urlRaw, requests.request(
            "GET", url, headers = headers, params = query_string_parameters
        )
    def searchElement(self, json, key):
        if key == 'Rating':
            try:
                return json['evaluation']['starRating']
            except KeyError:
                return np.nan
        elif key == 'Selling':
            try:
                lista = json['sellingPoints']
                flag = 0
                for i in range(0, len(lista)):
                    try:
                        flag += 1 if lista[i]['tagContent']['tagText'] == 'Envío gratis' else 0
                    except KeyError:
                        pass
                return True if flag > 0 else False
            except KeyError:
                return False
        elif key == 'Trade':
            try:
                return int(json['trade']['tradeDesc'].replace(' vendido(s)', ''))
            except KeyError:
                return np.nan
    def getValues(self, dict):
        rows = dict['mods']['itemList']['content']
        Products = []
        for i in range(0, len(rows)):
            ID     = rows[i]['productId']
            Rating = self.searchElement(rows[i], key = 'Rating')
            Trade  = self.searchElement(rows[i], key = 'Trade')
            Envio  = self.searchElement(rows[i], key = 'Selling')
            Titulo = rows[i]['title']['displayTitle']
            Precio = float(rows[i]['prices']['salePrice']['formattedPrice'].replace('COP ', '').replace(',', ''))
            URL    = 'https://es.aliexpress.com/item/'+rows[i]['productId']+'.html'
            Products.append((ID, URL, Titulo, Precio, Rating, Trade, Envio))
        return Products
    def createDF(self, list):
        df = pd.DataFrame(list, columns = ['ID', 'URL', 'Producto', 'Precio', 'Ranting', 'Vendidos', 'EnvíoGratis'])
        df['Origen'] = "AliExpress"
        df.to_csv('Support Files/AliExpress.csv', index = False)
        df.to_csv('Support Files/AliExpress.txt', sep = '|', index = False)
        return df
    def controller(self):
        Values = []
        j = 1
        while True:
            Website, Result = self.getConnection(self.cleanPath(), j)
            if Result.status_code == 200:
                print(Website)
                Content = Result.json()
                try:
                    Info = self.getValues(Content)
                    Values.extend(Info)
                    if len(Values) >= int(self.limit):
                        Values = Values[:self.limit]
                        return self.createDF(Values)
                    else:
                        j += 1
                except KeyError:
                    Values = []
                    break
            else:
                print(f"Sin respuesta por parte del servidor. Fallo al tratar de visitar la página web:\n\t--> {Website}"
                )
                break

___
## **CONSOLIDADO DE RESULTADOS**

In [7]:
os.chdir(r'D:\Documentos\JOBS\DNPE\DocumentosDNPE\Web Scraping')

dfAll = pd.DataFrame(columns = [
    'Origen', 'ID', 'URL', 'Producto', 'Precio', 'Categoria',
    'Ranting', 'Vendidos', 'EnvíoGratis'
    ]
)

dfAll = pd.concat(
    [dfAll,
     WebScrapingEasy('Herramientas > Herramientas para Madera > Lijadoras').controller(),
     WebScrapingHomeCenter('Herramientas y Maquinarias > Herramientas Eléctricas e Inalámbricas > Lijadoras Eléctricas y Accesorios').controller(),
     WebScrapingMercadoLibre('lijadora', limit = 14, infoExtra = True).controller(),
     WebScrapingAliExpress('lijadora', limit = 10).controller()
    ], axis = 0)

dfAll['HoraEjecución'] = (datetime.now()).strftime('%d/%m/%Y %H:%M:%S')
dfAll.to_csv('Code/Consolidado.csv', index = False)
dfAll

Página Número: 1 | Link: https://www.easy.com.co/c/herramientas/herramientas-para-madera/lijadoras/
https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios/
Página Número: 1 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios/
Página Número: 2 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios?currentpage=2
Página Número: 3 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios?currentpage=3
Página Número: 4 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios?currentpage=4
Página Número: 5 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios?currentpage=5
Página Número: 6 | Link: https://www.homecenter.com.co/homecenter-co/category/cat300032/lijadoras-electricas-y-accesorios?currentpage=6

,Origen,ID,URL,Producto,Precio,Categoria,Ranting,Vendidos,EnvíoGratis,HoraEjecución
0,Easy,NaN,https://www.easy.com.co/p/lijadora-orbital-ina...,Lijadora Orbital Inalambrica 24000 Opm,299.990,NaN,NaN,NaN,NaN,14/01/2023 23:16:53
1,Easy,NaN,https://www.easy.com.co/p/lijadora-triangular-...,Lijadora Triangular Inalambrica 24000 (Solo Eq...,188.990,NaN,NaN,NaN,NaN,14/01/2023 23:16:53
2,Easy,NaN,https://www.easy.com.co/p/lijadora-stel401-orb...,Lijadora STEL401 Orbital 220W 16000rpm,237.900,NaN,NaN,NaN,NaN,14/01/2023 23:16:53
3,Easy,NaN,https://www.easy.com.co/p/lijadora-1~3-de-150-...,Lijadora 1/3 de 150 W de Potencia,119.990,NaN,NaN,NaN,NaN,14/01/2023 23:16:53
4,Easy,NaN,https://www.easy.com.co/p/lijadora-dwe6421-pal...,Lijadora DWE6421 Palma Orbital 280w 12000opm,614.900,NaN,NaN,NaN,NaN,14/01/2023 23:16:53
...,...,...,...,...,...,...,...,...,...,...
5,AliExpress,1005003217998748,https://es.aliexpress.com/item/100500321799874...,HYVST-LIJADORA Orbital aleatoria multifunción ...,814146.39,NaN,4.9,168.0,False,14/01/2023 23:16:53
6,AliExpress,1005004296565077,https://es.aliexpress.com/item/100500429656507...,"Máquina pulidora de belleza para coche, lijado...",268830.95,NaN,5.0,22.0,True,14/01/2023 23:16:53
7,AliExpress,1005005120508701,https://es.aliexpress.com/item/100500512050870...,"LIJADORA Orbital neumática de pulgadas, amolad...",246109.47,NaN,NaN,NaN,False,14/01/2023 23:16:53
8,AliExpress,1005003994256463,https://es.aliexpress.com/item/100500399425646...,"Mini lijadora de 110V-220V, 18W, modelo de máq...",112363.68,NaN,5.0,148.0,False,14/01/2023 23:16:53
